ΜΕ ΧΡΗΣΗ ΕΝΟΣ (1) ΠΑΣΟΥ ΓΙΑ ΠΑΡΑΤΑΣΗ ΕΩΣ 13/1

# Βήμα 1

In [34]:
import pandas as pd

csv_file = pd.read_csv('cell_phone_ratings.csv')  # Διάβασμα αρχείου CSV


while True:
    user_counts = csv_file['reviewerID'].value_counts()  # Υπολογισμός του αριθμού των βαθμολογιών ανά χρήστη
    phone_counts = csv_file['asin'].value_counts()  # Υπολογισμός του αριθμού των βαθμολογιών ανά τηλέφωνο

    filtered_users = user_counts[user_counts > 5] # χρήστες που έχουν κάνει τουλάχιστον 5 αξιολογήσεις   
    filtered_phones = phone_counts[phone_counts > 5] # κινητά με τουλάχιστόν 5 αξιολογήσεις 
    
    # Νέο DataFrame, το οποίο περιέχει μόνο τις γραμμές από το αρχικό DataFrame που αντιστοιχούν στους χρήστες και τα τηλέφωνα που έχουν περάσει τα κριτήρια φιλτραρίσματος
    filtered_data = csv_file[csv_file['reviewerID'].isin(filtered_users.index) & csv_file['asin'].isin(filtered_phones.index)]
    
    # Έλεγχος αν όλοι οι χρήστες έχουν αξιολογήσει τουλάχιστον 5 φορές και αν ολα τα κινητα έχουν τουλαχιστον 5 αξιολογήσεις
    if len(csv_file)==len(filtered_data):
        break
    csv_file = filtered_data

print("Num of filtered data:", len(csv_file))
print("Num of users that have reviewed at least 5 cellphones:", len(filtered_users))
print("Num of phones that have been reviewed from at least 5 users:", len(filtered_phones))


Num of filtered data: 2969
Num of users that have reviewed at least 5 cellphones: 389
Num of phones that have been reviewed from at least 5 users: 177


In [35]:
from sklearn.model_selection import train_test_split

# Δημιουργία train και test sets
train_data, test_data = train_test_split(filtered_data, test_size=0.1, train_size=0.9, random_state=43)

print("Αριθμός τριάδων στο train set:", len(train_data))
print("Αριθμός τριάδων στο test set:", len(test_data))

Αριθμός τριάδων στο train set: 2672
Αριθμός τριάδων στο test set: 297


# Βήμα 2

In [85]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt


ratings_test = pd.read_csv('ratings_test.csv')  # Διάβασμα αρχείου CSV
ratings_train = pd.read_csv('ratings_train.csv')  # Διάβασμα αρχείου CSV

UA_ratings = ratings_train.groupby('reviewerID')['overall'].mean() # Υπολογισμός User Average

IA_ratings = ratings_train.groupby('asin')['overall'].mean() # Υπολογισμός Item Average

# Συνάρτηση για πρόβλεψη βαθμολογίας με βάση τον User Average
def predict_UA(user):
    return UA_ratings.loc[user] # επιστρέφει τον μέσο όρο (βαθμολογία) του χρήστη από το UA_ratings Dataframe 
    #(με την εντολή .loc(user) επιλέγεται η σειρά που αντιστοιχεί στον συγκεκριμένο χρήστη από το Dataframe UA_ratings)

    
# Συνάρτηση για πρόβλεψη βαθμολογίας με βάση τον Item Average
def predict_IA(item):
    return IA_ratings.loc[item] # επιστρέφει τον μέσο όρο (βαθμολογία) του item από το IA_ratings Dataframe 
    
    
# Προβλέψεις για το test set
ratings_test['predicted_UA'] = ratings_test['reviewerID'].apply(predict_UA)
ratings_test['predicted_IA'] = ratings_test['asin'].apply(predict_IA)

# Υπολογισμός RMSE για User Average
rmse_UA = sqrt(mean_squared_error(ratings_test['overall'], ratings_test['predicted_UA']))

# Υπολογισμός RMSE για Item Average
rmse_IA = sqrt(mean_squared_error(ratings_test['overall'], ratings_test['predicted_IA']))

print("Το RMSE για τον αλγόριθμο User Average (UA):", rmse_UA)
print("Το RMSE για τον αλγόριθμο Item Average (IA):", rmse_IA)


Το RMSE για τον αλγόριθμο User Average (UA): 0.5397865558697561
Το RMSE για τον αλγόριθμο Item Average (IA): 0.5138120657943773


# Βήμα 3

In [58]:
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd


# Διάβασμα των training data και test data από τα CSV
train_data = pd.read_csv('ratings_train.csv')
test_data = pd.read_csv('ratings_test.csv')

# Αντιστοίχιση reviewerIDs
unique_reviewerIDs = train_data['reviewerID'].unique()
reviewerID_mapping = {id: i for i, id in enumerate(unique_reviewerIDs)}

train_data['reviewerID_mapped'] = train_data['reviewerID'].map(reviewerID_mapping)

# Αντιστοίχιση asins
unique_asins = train_data['asin'].unique()
asin_mapping = {asin: i for i, asin in enumerate(unique_asins)}

train_data['asin_mapped'] = train_data['asin'].map(asin_mapping)

# Δημιουργία ενός αραιού πίνακα γεμισμένου με μηδενικά
sparse_array = np.zeros((len(train_data['reviewerID'].unique()), len(train_data['asin'].unique())))

# Διατρέχουμε όλες τις γραμμές της στήλης reviewerID και asin
# Με την χρήση του .items() επιστρέφεται ένα ζεύγος (index,value), όπου index είναι ο αριθμός γραμμής και το value είναι η τιμή στη συγκεκριμένη στήλη για αυτήν τη γραμμή.
for (reviewerID_mapping, reviewerID), (asin_mapping, asin) in zip(train_data['reviewerID'].items(), train_data['asin'].items()):
    # Ενημέρωση του αραιού πίνακα
    sparse_array = pd.crosstab(train_data['reviewerID_mapped'], train_data['asin_mapped'])


# Υπολογισμός της SVD
U, Sigma, VT = np.linalg.svd(sparse_array, full_matrices=False)

# Πρώτα 20 singular values και τις αντίστοιχες στήλες των U και VT
k = 20
U_20 = U[:, :k]
Sigma_20 = np.diag(Sigma[:k])
VT_20 = VT[:k, :]

# Προσέγγιση rank-20 του πίνακα sparse_array
R_20 = U_20 @ Sigma_20 @ VT_20

# Πρώτα 20 singular values
singular_values_20 = Sigma[:k]

# Εκτύπωση των πρώτων 20 singular values
print("Πρώτα 20 Singular Values:", singular_values_20)


# Μέχρι αυτό το σημείο ο κώδικας τρέχει χωρίς κάποιο σφάλμα
# Παρακάτω παραθέτω το σκεπτικό για την υλοποίηση του υπόλοιπου βήματος, αλλα΄ σε σχόλια καθ΄ώς υπάρχει κάποιο error που λόγω χρόνου δεν επιλύθηκε

#rmse_values = []  # Λίστα για την αποθήκευση των τιμών RMSE για κάθε k

#for k in range(1, K + 1):
    # Επιλογή των πρώτων k συνιστωσών
#    U_k = U[:, :k]
#    Sigma_k = np.diag(Sigma[:k])
#    VT_k = VT[:k, :]

    # Υπολογισμός των τιμών Rk[u, c]
#    predictions = np.array([np.dot(U_k[i, :], np.dot(Sigma_k, VT_k[:, j])) for i, j in zip(test_data['reviewerID'], test_data['asin'])])

    # Υπολογισμός του RMSE
#    rmse = sqrt(mean_squared_error(test_data['ratings'], predictions))
#    rmse_values.append(rmse)

# Εκτύπωση των τιμών RMSE για κάθε k
#for k, rmse_value in enumerate(rmse_values, 1):
#    print(f'k={k}: RMSE={rmse_value}')
    
# Γραφική παράσταση του RMSE ως προς το k
#plt.plot(K_values, RMSE_values, marker='o')
#plt.xlabel('k')
#plt.ylabel('RMSE')
#plt.title('RMSE for different values of k')
#plt.show()

 #Εύρεση της ελάχιστης τιμής του RMSE και του αντίστοιχου k
#min_rmse = min(RMSE_values)
#optimal_k = K_values[RMSE_values.index(min_rmse)]

#print(f"The optimal value of k is {optimal_k} with RMSE {min_rmse}")



Πρώτα 20 Singular Values: [16.44491525 10.7059381   9.10408467  8.54050128  8.26006579  7.4454296
  7.2426423   6.87369706  6.72589658  6.67790062  6.53007773  6.49126436
  6.41190968  6.2810104   6.16455132  6.09859499  6.04507528  5.91376564
  5.85964205  5.80882594]
